In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt # general plotting
import cartopy.crs as ccrs # plot on maps, better than the Basemap module
import re
from scipy.interpolate import interp1d

In [2]:
ds = xr.open_dataset(".././initial_GEOSChem_rst.c48_benchmark.nc")
ds # same as print(ds) in IPython/Jupyter environment

/Users/adashaw/anaconda3/lib/python3.6/site-packages/xarray/conventions.py:537: SerializationWarning: variable 'SPC_RCOOH' has multiple fill values {-1e+31, -1e-31}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/adashaw/anaconda3/lib/python3.6/site-packages/xarray/conventions.py:537: SerializationWarning: variable 'SPC_O2' has multiple fill values {-1e+31, -1e-31}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/adashaw/anaconda3/lib/python3.6/site-packages/xarray/conventions.py:537: SerializationWarning: variable 'SPC_N2' has multiple fill values {-1e+31, -1e-31}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/adashaw/anaconda3/lib/python3.6/site-packages/xarray/conventions.py:537: SerializationWarning: variable 'SPC_MOH' has multiple fill values {-1e+31, -1e-31}, decoding all values to NaN.
  stack_char_dim=stack_char_dim)
/Users/adashaw/anaconda3/lib/python3.6/site-packages/xarray/conventions.py:537: SerializationWarning

<xarray.Dataset>
Dimensions:       (lat: 288, lev: 72, lon: 48, time: 1)
Coordinates:
  * lon           (lon) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * lat           (lat) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * lev           (lev) float64 72.0 71.0 70.0 69.0 68.0 67.0 66.0 65.0 64.0 ...
  * time          (time) datetime64[ns] 2013-07-01
Data variables:
    SPC_RCOOH     (time, lev, lat, lon) float32 ...
    SPC_O2        (time, lev, lat, lon) float32 ...
    SPC_N2        (time, lev, lat, lon) float32 ...
    SPC_MOH       (time, lev, lat, lon) float32 ...
    SPC_H2        (time, lev, lat, lon) float32 ...
    SPC_O         (time, lev, lat, lon) float32 ...
    SPC_HO2       (time, lev, lat, lon) float32 ...
    SPC_O1D       (time, lev, lat, lon) float32 ...
    SPC_OH        (time, lev, lat, lon) float32 ...
    SPC_MO2       (time, lev, lat, lon) float32 ...
    SPC_MCO3      (time, lev, lat, lon) float32 ...
    SPC_MRO2      (time, lev, lat, lon) 

In [3]:
lev = np.linspace(1,132,num =132,dtype = 'float64')
molec = list()
i = 0
pattern = "(SPC_\w+)"
for i, line in enumerate(open('../molecular_species.txt')):
    for match in re.finditer(pattern,line):
        #molec.append(match)
        molec.append(match.group())


In [4]:
dataset = xr.Dataset()
dataset.coords['time'] = ds.coords['time']
dataset.coords['lev'] = (('lev'),lev)
dataset.coords['lat'] = ds.coords['lat'] 
dataset.coords['lon'] = ds.coords['lon']
dataset.attrs['title'] = 'GEOS-5 132 lvl restart'
dataset.attrs['history'] = 'created by Ada Shaw with initial_GEOSChem_rst.c48_benchmark.nc'
dataset.attrs['format'] = "NetCDF-4"
dataset.attrs['conventions'] = 'COARDS'

In [5]:
for tick_molec in range(len(molec)):
    dr = ds[molec[tick_molec]].values
    #dr.shape # time:1, lev:72, lat:46, lon:72
    #initialize x_interp array to be 1x132x46x72
    type(dr)
    x_interp = np.ndarray(shape=(dr.shape[0],132,dr.shape[2],dr.shape[3]), dtype='f', order='F')
    for tick_lat in range(dr.shape[2]):
        for tick_lon in range(dr.shape[3]):
            xo = dr[0,0,tick_lat,tick_lon]
            xf = dr[0,dr.shape[1]-1,tick_lat,tick_lon]
            x_interp[0,:,tick_lat,tick_lon]=np.linspace(xo,xf,num=132,dtype = 'f')
    dataset[molec[tick_molec]] = (('time','lev','lat','lon'),x_interp)
    dataset.data_vars[molec[tick_molec]].attrs = ds.data_vars[molec[tick_molec]].attrs

In [6]:
dataset

<xarray.Dataset>
Dimensions:       (lat: 288, lev: 132, lon: 48, time: 1)
Coordinates:
  * time          (time) datetime64[ns] 2013-07-01
  * lev           (lev) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * lat           (lat) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
  * lon           (lon) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...
Data variables:
    SPC_RCOOH     (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_O2        (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_N2        (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_MOH       (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_H2        (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_O         (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_HO2       (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC_O1D       (time, lev, lat, lon) float32 1e-30 1e-30 1e-30 1e-30 ...
    SPC

In [7]:
dataset.to_netcdf('132_lvl_rst.nc','w')